# Part B - Merchant Churn Analysis

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

### Step1 - Data Preparation

In [34]:
#Loading the data set
df = pd.read_csv('takehome_ds_written.csv')
df.head()

,Unnamed: 0,merchant,time,amount_usd_in_cents
0,1,faa029c6b0,2034-06-17 23:34:14,6349
1,2,ed7a7d91aa,2034-12-27 00:40:38,3854
2,3,5608f200cf,2034-04-30 01:29:42,789
3,4,15b1a0d61e,2034-09-16 01:06:23,4452
4,5,4770051790,2034-07-22 16:21:42,20203


In [35]:
df = df.drop(columns="Unnamed: 0")
df["amount_usd"] = df["amount_usd_in_cents"]/100
df = df.drop(columns="amount_usd_in_cents")
df["time"] = pd.to_datetime(df["time"])
df.head()

,merchant,time,amount_usd
0,faa029c6b0,2034-06-17 23:34:14,63.49
1,ed7a7d91aa,2034-12-27 00:40:38,38.54
2,5608f200cf,2034-04-30 01:29:42,7.89
3,15b1a0d61e,2034-09-16 01:06:23,44.52
4,4770051790,2034-07-22 16:21:42,202.03


### Step2 - Defining Churn and model for identifying merchant churn

In [36]:
#No of days between consecutive transactions of each merchant in day format "days_btwn_consecutive"
df2 = df.sort_values(["merchant","time"]).reset_index(drop=True)
df2['days_btwn_consecutive'] = df2.groupby("merchant")["time"].diff()
df2["days_btwn_consecutive"] = df2["days_btwn_consecutive"].dt.days

In [37]:
df2.head()

,merchant,time,amount_usd,days_btwn_consecutive
0,0002b63b92,2033-05-16 20:07:57,33.79,NaN
1,0002d07bba,2034-10-11 17:02:26,55.49,NaN
2,0002d07bba,2034-10-17 17:57:32,378.30,6.0
3,0002d07bba,2034-11-13 15:42:55,438.42,26.0
4,0002d07bba,2034-12-15 09:56:19,20.57,31.0


In [38]:
#Data frame consisting of each merchant and their respective last transaction
df_last = df2.groupby("merchant")["time"].tail(1).to_frame()
#Tagging each merchant to show if the 'days_btwn_consecutive' is >180 days in 'consec_flag180'
df_last["consec_flag180"] = np.where(df2.groupby("merchant")["days_btwn_consecutive"].max() >= 180, '>180', '<180')

In [39]:
#Adding merchant and the last transaction in the dataset to the data frame df-last
df_last["merchant"] = df2["merchant"].unique()
df_last["last_transaction"] = df.sort_values("time").iloc[-1,1]

In [40]:
#No of days between each merchant's last transaction and the end date in the data set 'days_btwn_lasttransact'
df_last["days_btwn_lasttransact"] = df_last["last_transaction"] - df_last["time"]
df_last["days_btwn_lasttransact"] = df_last["days_btwn_lasttransact"].dt.days

In [41]:
#Tagging each merchant to show if the 'days_btwn_consecutive' is >180 days in 'consec_flag180'
df_last["lasttransact_flag180"] = np.where(df_last["days_btwn_lasttransact"] >= 180, '>180', '<180')
df_last.head()

,time,consec_flag180,merchant,last_transaction,days_btwn_lasttransact,lasttransact_flag180
0,2033-05-16 20:07:57,<180,0002b63b92,2034-12-31 07:59:40,593,>180
4,2034-12-15 09:56:19,<180,0002d07bba,2034-12-31 07:59:40,15,<180
32,2033-08-04 04:26:40,<180,00057d4302,2034-12-31 07:59:40,514,>180
33,2033-08-09 20:18:36,<180,000bcff341,2034-12-31 07:59:40,508,>180
34,2033-06-02 13:25:12,<180,000ddbf0ca,2034-12-31 07:59:40,576,>180


In [43]:
#No of merchants that were inactive for 180 days but came back eventualy in the given time period
returning = df_last.loc[df_last["consec_flag180"]==">180"].shape[0]
returning

1504

In [44]:
#No of merchants that we can call churned based on the churn period selected (180 days)
churned = df_last.loc[(df_last["consec_flag180"]=="<180") & (df_last["lasttransact_flag180"]==">180")].shape[0]
churned

3856

In [45]:
#No of merchants that we can call say have not churned based on the churn period selected (180 days)
df_last.loc[(df_last["consec_flag180"]=="<180") & (df_last["lasttransact_flag180"]=="<180")].shape[0]

8991

In [53]:
#Merchant churn percentage under the churn period 180 days consideration
churn_percentage = churned / (churned+returning) *100
churn_percentage

71.94029850746269

In [52]:
#List of merchants that have churned
df_churn = df_last.loc[(df_last["consec_flag180"] == "<180") & (df_last["lasttransact_flag180"] == ">180")]
df_churn["merchant"]

0          0002b63b92
32         00057d4302
33         000bcff341
34         000ddbf0ca
387        001f7b7a4f
              ...    
1513492    ffc46fd720
1513494    ffc5a319bb
1513535    ffd3e45675
1513588    ffe1f6b51a
1513672    ffec05edb9
Name: merchant, Length: 3856, dtype: object